In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!unzip /content/drive/MyDrive/Origin/images-20230823T141023Z-001.zip -d /content/drive/MyDrive/Origin/data

Archive:  /content/drive/MyDrive/Origin/images-20230823T141023Z-001.zip
  inflating: /content/drive/MyDrive/Origin/data/images/Patient01447_Plane5_1_of_1.png  
  inflating: /content/drive/MyDrive/Origin/data/images/Patient01702_Plane3_5_of_8.png  
  inflating: /content/drive/MyDrive/Origin/data/images/Patient01162_Plane6_4_of_4.png  
  inflating: /content/drive/MyDrive/Origin/data/images/Patient01210_Plane3_1_of_3.png  
  inflating: /content/drive/MyDrive/Origin/data/images/Patient01112_Plane5_1_of_2.png  
  inflating: /content/drive/MyDrive/Origin/data/images/Patient00704_Plane3_1_of_5.png  
  inflating: /content/drive/MyDrive/Origin/data/images/Patient01287_Plane3_1_of_1.png  
  inflating: /content/drive/MyDrive/Origin/data/images/Patient00786_Plane6_4_of_5.png  
  inflating: /content/drive/MyDrive/Origin/data/images/Patient01529_Plane6_1_of_2.png  
  inflating: /content/drive/MyDrive/Origin/data/images/Patient01423_Plane6_1_of_2.png  
  inflating: /content/drive/MyDrive/Origin/data/

In [13]:
import os
import shutil

# Source directory containing the images
destination_dir = "/content/drive/MyDrive/Origin/Images"

source_dir = "/content/drive/MyDrive/Origin/data/images"


# Define the mapping of categories
categories = {
    "Plane3": 556,
    "Plane5": 300,
    "Plane6": 450,
    "Plane2": 332
}

def rename_images():
    image_count = {}
    for filename in os.listdir(source_dir):
        if filename.endswith(".png"):  # Adjust the extension as needed
            parts = filename.split("_")
            if len(parts) >= 2:
                plane_info = parts[1]
                if plane_info in categories:
                    if plane_info not in image_count:
                        image_count[plane_info] = 1
                    new_filename = f"{plane_info}_{image_count[plane_info]}.png"
                    source_path = os.path.join(source_dir, filename)
                    destination_path = os.path.join(destination_dir, new_filename)
                    shutil.copy(source_path, destination_path)
                    print(f"Renamed '{filename}' to '{new_filename}'")
                    image_count[plane_info] += 1
                    categories[plane_info] -= 1
                    if categories[plane_info] == 0:
                        del categories[plane_info]
            else:
                print(f"Ignoring '{filename}' as it doesn't match the pattern.")

        if not categories:
            break  # Stop if all categories are processed

if __name__ == "__main__":
    rename_images()

Renamed 'Patient01447_Plane5_1_of_1.png' to 'Plane5_1.png'
Renamed 'Patient01702_Plane3_5_of_8.png' to 'Plane3_1.png'
Renamed 'Patient01162_Plane6_4_of_4.png' to 'Plane6_1.png'
Renamed 'Patient01210_Plane3_1_of_3.png' to 'Plane3_2.png'
Renamed 'Patient01112_Plane5_1_of_2.png' to 'Plane5_2.png'
Renamed 'Patient00704_Plane3_1_of_5.png' to 'Plane3_3.png'
Renamed 'Patient01287_Plane3_1_of_1.png' to 'Plane3_4.png'
Renamed 'Patient00786_Plane6_4_of_5.png' to 'Plane6_2.png'
Renamed 'Patient01529_Plane6_1_of_2.png' to 'Plane6_3.png'
Renamed 'Patient01423_Plane6_1_of_2.png' to 'Plane6_4.png'
Renamed 'Patient01097_Plane3_1_of_2.png' to 'Plane3_5.png'
Renamed 'Patient00944_Plane6_2_of_4.png' to 'Plane6_5.png'
Renamed 'Patient00864_Plane6_1_of_2.png' to 'Plane6_6.png'
Renamed 'Patient01654_Plane2_1_of_1.png' to 'Plane2_1.png'
Renamed 'Patient00947_Plane6_1_of_1.png' to 'Plane6_7.png'
Renamed 'Patient00806_Plane3_1_of_4.png' to 'Plane3_6.png'
Renamed 'Patient01132_Plane6_1_of_1.png' to 'Plane6_8.pn

In [17]:
import pandas as pd

# Read the csv file
df = pd.read_csv('/content/drive/MyDrive/Origin/image_label.csv')

# Create a new column with the desired format
df['new_image_name'] = df['Image_name'].apply(lambda x: x.split('_')[1])

# Create a dictionary to keep track of the count for each plane type
plane_count = {}

# Iterate over the rows of the dataframe
for i, row in df.iterrows():
    # Get the plane type for the current row
    plane_type = row['new_image_name']

    # If the plane type is not in the dictionary, add it with a count of 1
    if plane_type not in plane_count:
        plane_count[plane_type] = 1
    # Otherwise, increment the count for that plane type
    else:
        plane_count[plane_type] += 1

    # Update the new image name with the count
    df.at[i, 'new_image_name'] = f"{plane_type}_{plane_count[plane_type]}"

# Save the updated dataframe to a new csv file
df.to_csv('updated_file.csv', index=False)


In [19]:
import pandas as pd

# Load the CSV file into a DataFrame
csv_file_path = '/content/drive/MyDrive/Origin/updated_file.csv'
df = pd.read_csv(csv_file_path)

# Add ".png" to the values in the "Image_name" column
df['Image_name'] = df['Image_name'].apply(lambda x: x + '.png')

# Save the modified DataFrame back to the CSV file
df.to_csv(csv_file_path, index=False)

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

# Data Collection:
labels = pd.read_csv('/content/drive/MyDrive/Origin/updated_file.csv')
images_dir = '/content/drive/MyDrive/Origin/Images'

# Data Split: Divide the dataset into training, validation, and test sets.
train_data, test_data = train_test_split(labels, test_size=0.2, stratify=labels['Plane'])
train_data, val_data = train_test_split(train_data, test_size=0.125, stratify=train_data['Plane'])

# Data Augmentation: Applying thr techniques such as rotation, flipping, zooming, and cropping to artificially increase the dataset size.
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=images_dir,
    x_col='Image_name',
    y_col='Plane',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = datagen.flow_from_dataframe(
    dataframe=val_data,
    directory=images_dir,
    x_col='Image_name',
    y_col='Plane',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=images_dir,
    x_col='Image_name',
    y_col='Plane',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 1131 validated image filenames belonging to 4 classes.
Found 162 validated image filenames belonging to 4 classes.
Found 327 validated image filenames belonging to 4 classes.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 18 invalid image filename(s) in x_col="Image_name". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 3 invalid image filename(s) in x_col="Image_name". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 2 invalid image filename(s) in x_col="Image_name". These filename(s) will be ignored.
  warnings.warn(


In [21]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten

# Architecture Choice: Start with well-established architectures like Convolutional Neural Networks (CNNs).
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the base model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Model Construction: The model was constructed in the previous step.

# Loss Function: Select an appropriate loss function for multi-class classification, such as categorical cross-entropy.
loss_fn = 'categorical_crossentropy'

# Optimizer: Choose an optimizer like Adam to update the model's weights during training.
optimizer = 'adam'

# Compile the model with the chosen loss function and optimizer
model.compile(loss=loss_fn, optimizer=optimizer, metrics=['accuracy'])

# Set up early stopping and model checkpointing
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# Fit the model on the training data
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/100
36/36 [==============================] - 935s 26s/step - loss: 1.8016 - accuracy: 0.5933 - val_loss: 0.5905 - val_accuracy: 0.7531
Epoch 2/100
36/36 [==============================] - 890s 25s/step - loss: 0.5081 - accuracy: 0.8453 - val_loss: 0.4710 - val_accuracy: 0.8395
Epoch 3/100
36/36 [==============================] - 927s 26s/step - loss: 0.4259 - accuracy: 0.8683 - val_loss: 0.5698 - val_accuracy: 0.8272
Epoch 4/100
36/36 [==============================] - 899s 25s/step - loss: 0.3819 - accuracy: 0.8851 - val_loss: 0.3574 - val_accuracy: 0.9136
Epoch 5/100
36/36 [==============================] - 887s 25s/step - loss: 0.3238 - accuracy: 0.9036 - val_loss: 0.4317 - val_accuracy: 0.8765
Epoch 6/100
36/36 [==============================] - 884s 25s/step - loss: 0.3409 - accuracy: 0.8930 - val_loss: 0.3562 - val_accuracy: 0.8827
Epoch 7/100
36/36 [==============================] - 881s 25s/step - loss: 0.2933 - accuracy: 0.9116 - val_loss: 0.3958 - val_accuracy: 0.8642

In [1]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Load the best performing model
model = load_model('best_model.h5')

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}')

# Make predictions on the test data
test_generator.reset()
y_pred = model.predict(test_generator)
y_pred = np.argmax(y_pred, axis=1)

# Calculate the confusion matrix
cm = confusion_matrix(test_generator.classes, y_pred)
print('Confusion matrix:\n', cm)

# Calculate classification metrics
cr = classification_report(test_generator.classes, y_pred, target_names=test_generator.class_indices.keys())
print('Classification report:\n', cr)


OSError: ignored

In [ ]:
# Model Improvement: Analyze the model's performance and error cases.

# Fine-tune the model by unfreezing some of the base model layers
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-5), metrics=['accuracy'])

# Continue training the model with the fine-tuned layers
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=[early_stopping, model_checkpoint]
)

Testing on External Test Images

In [ ]:
from keras.preprocessing import image
import numpy as np
import os

# Load the best performing model
model = load_model('best_model.h5')

# Set the path to the directory containing the external test images
test_images_dir = '/content/drive/MyDrive/Origin/External Test images'

# Loop over the test images
for filename in os.listdir(test_images_dir):
    # Load the image and preprocess it
    img_path = os.path.join(test_images_dir, filename)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0

    # Make a prediction using the trained model
    y_pred = model.predict(x)
    y_pred_class = np.argmax(y_pred, axis=1)

    # Map the predicted class index to the class name
    class_names = ['Fetal abdomen', 'Fetal brain', 'Fetal femur', 'Fetal thorax']
    y_pred_class_name = class_names[y_pred_class[0]]

    # Print the prediction result
    print(f'Image: {filename}, Predicted class: {y_pred_class_name}')
